### Setup

In [1]:
# install if not yet done
using Pkg

# Pkg.add(url="https://github.com/mppmu/BoostFractor.jl.git")
# Pkg.add(url="https://github.com/bergermann/Dragoon.jl.git")
Pkg.update()

    Updating registry at `C:\Users\bergermann\.julia\registries\General.toml`
    Updating git-repo `https://git.rwth-aachen.de/nick1/KeyVNA-jl.git`
    Updating git-repo `https://git.rwth-aachen.de/nick1/XIMC-jl.git`
    Updating git-repo `https://github.com/mppmu/BoostFractor.jl.git`
    Updating git-repo `https://github.com/bergermann/Dragoon.jl.git`
   Installed OffsetArrays ───────── v1.17.0
   Installed Libmount_jll ───────── v2.41.0+0
   Installed Git_jll ────────────── v2.49.0+0
   Installed GR_jll ─────────────── v0.73.14+0
   Installed Xorg_xkbcomp_jll ───── v1.4.7+0
   Installed Roots ──────────────── v2.2.7
   Installed HTTP ───────────────── v1.10.16
   Installed Xorg_libSM_jll ─────── v1.2.6+0
   Installed Cairo_jll ──────────── v1.18.4+0
   Installed Fontconfig_jll ─────── v2.16.0+0
   Installed Xorg_libxkbfile_jll ── v1.1.3+0
   Installed Xorg_libXinerama_jll ─ v1.1.6+0
   Installed Xorg_libXau_jll ────── v1.0.13+0
   Installed JLD2 ───────────────── v0.5.13
   Installe

In [2]:
using Dragoon
using Plots

In [6]:
n = 4 # number of discs in the booster

initdist = findpeak1d(25.025e9,n) # initial disc configuration

freqs = genFreqs(25.025e9,50e6; n=10) # optimize on these frequencies
freqsplot = genFreqs(25.025e9,150e6; n=1000) # plot with these frequencies

booster = AnalyticalBooster(initdist; ndisk=n,ϵ=9.,τ=2e-3)  # physical properties of the booster
booster.wavelength = λ(25.025e9)

hist = initHist(booster,1000,freqs,ObjAnalytical); # stores measurements
#                           ^ length, should be at least n

1000-element Vector{State}:
 State([0.00608759173973974, 0.01317518347947948, 0.020262775219219223, 0.02735036695895896], -33.96723527149639, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 ⋮
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 0.0, 0.0, 0.0], 0.0, Dates.DateTime("0000-01-01T00:00:00"))
 State([0.0, 

### Optimizer

In [7]:
trace = nelderMead(booster,hist,freqs,
                    1.,1+2/n,0.75-1/2n,1-1/n,1e-9,
                    ObjAnalytical,
                    InitSimplexRegular(1e-4),
                    DefaultSimplexSampler,
                    UnstuckDont;
                    maxiter=Int(1e5),
                    showtrace=true,
                    showevery=100,
                    unstuckisiter=true,)

MethodError: MethodError: no method matching nelderMead(::AnalyticalBooster, ::Vector{State}, ::Vector{Float64}, ::Float64, ::Float64, ::Float64, ::Float64, ::Float64, ::Callback, ::Callback, ::Callback, ::Callback; maxiter::Int64, showtrace::Bool, showevery::Int64, unstuckisiter::Bool)
The function `nelderMead` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  nelderMead(::Booster, ::Vector{State}, ::Array{Float64}, ::Float64, ::Float64, ::Float64, ::Float64, ::Real, !Matched::Real, ::Callback, ::Callback, ::Callback, !Matched::Callback; maxiter, showtrace, showevery, unstuckisiter, forcesimplexobj, traceevery, resettimer, returntimes)
   @ Dragoon C:\Users\bergermann\.julia\packages\Dragoon\VVERx\src\neldermead\optimizer.jl:48


In [ ]:
# obtain result, booster is now in optimized position
pos2dist(booster.pos)     #in distance space
# booster.pos               #in position space

In [ ]:
typeof(trace)

In [ ]:
# get handy output
plt = analyse(booster,hist,trace,freqsplot; freqs=freqs,div=10,ylim=[0,100e3])

# savefig(plt[i],"cool_epic_result_wow.svg")

In [ ]:
booster.timestamp

In [ ]:
analyse()